## Classes and methods for plotting x-ray diffraction comparisons

In [ ]:
class PowderAfterElectrolysis:
    
    # Name used for graphene
    grapheneName = "Graphene"
    
    # constructor
    def __init__(self, oxideFormula, electrolyteFormula, electrodeFormula, backgroundColor, themeColor):
        self.oxideFormula = oxideFormula
        self.electrolyteFormula = electrolyteFormula
        self.electrodeFormula = electrodeFormula
        self.backgroundColor = backgroundColor
        self.themeColor = themeColor
        
    # get the powder name as a combination of graphene and metal oxide (does not include electrolyte)    
    def powderFormula(self):
        return self.grapheneName + "/" + self.oxideFormula

In [1]:
import csv
import matplotlib.pyplot as plt
from scipy.misc import imread
import numpy as np
from PIL import Image
import os.path

def normalize(list_obj, multiplier, offset):
    
    # convert to floats, otherwise it has troubles handling '5.000' format with min and max functions
    list_obj = map(float, list_obj)
    
    # get min and max values from the list
    min_val = min(list_obj)
    max_val = max(list_obj)
    
    # return normalized list with multiplier and offset applied
    return [multiplier * (n-min_val) / (max_val-min_val) + offset for n in list_obj]
    
def plotBeforeAfterHeating(powder, isLog, save, plotsColors):

    # Choose if you should show data logarithmically scaled
    log10sufixes = ['', '_log10'] 
    if isLog:
        log10sufix = log10sufixes[1]
    else:
        log10sufix = log10sufixes[0]

    # Choose whether to show the figure inline or save to the computer
    toSave = save

    # Oxide's chemical formula 
    oxide = powder.oxideFormula
    chloride = powder.electrolyteFormula

    # Axis limits, fixed in order to display background image properly
    axisLimits = [0, 100, 0, 70]

    fileNames = [oxide + chloride, oxide + chloride + '_550', oxide + chloride + '_650']
    # , oxide + chloride + '_before'
    
    # color used for title, legend, ticks, spines etc.
    themeColor = powder.themeColor
    background_color = powder.backgroundColor
    
    fig=plt.figure(facecolor=background_color)

    # Remove ticks from y axis
    ax = fig.add_subplot(111)
    ax.set_yticklabels([])
    plt.setp(ax.spines.values(), color=themeColor)
    ax.xaxis.label.set_color(themeColor)
    ax.yaxis.label.set_color(themeColor)
    ax.tick_params(colors=themeColor)

    for fn in range(len(fileNames)):
        x, y = [], []
        data_path = 'xrd/' + fileNames[fn] + '.xy'

        if os.path.isfile(data_path):
            with open(data_path,'r') as f:
                reader = csv.reader(f, delimiter='\t')
                for row in reader:
                    x.append(row[0])
                    y.append(row[1])

            # scale results depending on whether it is log or not
            if log10sufix:        
                y = [np.log10(float(i)) for i in y]

            multiplier = 24
            if fileNames[fn] == 'ZnOFeCl3':
                multiplier = 12
            # normalize and make of
            y = normalize(y, multiplier, 18 * fn + 5)

            # Background image for the plot
            image_path = "SEM/" + oxide + chloride + ".tif"
            if os.path.isfile(image_path):
                image=Image.open(image_path)
                plt.imshow(image, zorder=0, extent=axisLimits).set_alpha(0.2)

        # plot results
        plt.plot(x,y,plotsColors[fn])

    # Plot properties    
    plt.title(powder.powderFormula() + ' in ' + powder.electrolyteFormula + ' electrolyte with ' 
              + powder.electrodeFormula + ' electrode', color=themeColor)
    plt.xlabel(r'$2 \theta$ [deg]', fontsize=10)
    plt.ylabel('Intensity' + log10sufix + ' [Counts, arb.]', fontsize=10)
    plt.axis(axisLimits)
    leg = plt.legend(['Before heating', 'After heating in 550 C', 'After heating in 650 C'])
    leg.get_frame().set_alpha(0.5) # set the alpha value of the legend: it will be translucent
    for text in leg.get_texts():
        text.set_color(background_color)

    # show or save plot
    if toSave:
        plt.savefig('figures/before and after heating/Graphene_' + oxide + '_' + chloride + log10sufix + '.png', 
                    dpi=200, facecolor=fig.get_facecolor(), transparent=True)
    else:
        plt.show()

    # clear and close figures
    plt.clf()
    plt.close() 

SyntaxError: invalid syntax (<ipython-input-1-52e6ea2edf2a>, line 72)